# Gensim tests

In [ ]:
import logging

In [ ]:
from gensim import corpora, models, similarities

In [ ]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## Preliminaries

In [ ]:
corpus = [[(0, 1.0), (1, 1.0), (2, 1.0)],
          [(2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0), (6, 1.0), (8, 1.0)],
          [(1, 1.0), (3, 1.0), (4, 1.0), (7, 1.0)],
          [(0, 1.0), (4, 2.0), (7, 1.0)],
          [(3, 1.0), (5, 1.0), (6, 1.0)],
          [(9, 1.0)],
          [(9, 1.0), (10, 1.0)],
          [(9, 1.0), (10, 1.0), (11, 1.0)],
          [(8, 1.0), (10, 1.0), (11, 1.0)]]

In [ ]:
tfidf = models.TfidfModel(corpus)

In [ ]:
vec = [(0, 1), (4, 1)]
print(tfidf[vec])

In [ ]:
index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features=12)

In [ ]:
sims = index[tfidf[vec]]
print(list(enumerate(sims)))

## Tutorial 1 : Corpora and Vector Spaces

In [ ]:
documents = ["Human machine interface for lab abc computer applications",
             "A survey of user opinion of computer system response time",
             "The EPS user interface management system",
             "System and human system engineering testing of EPS",
             "Relation of user perceived response time to error measurement",
             "The generation of random binary unordered trees",
             "The intersection graph of paths in trees",
             "Graph minors IV Widths of trees and well quasi ordering",
             "Graph minors A survey"]

In [ ]:
from collections import defaultdict
from pprint import pprint  # pretty-printer

# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in documents]

# remove words that appear only once
frequency = defaultdict(int)

for text in texts:
    for token in text:
        frequency[token] += 1
texts = [[token for token in text if frequency[token] > 1]
         for text in texts]

pprint(texts)

In [ ]:
dictionary = corpora.Dictionary(texts)
dictionary.save('../tmp/deerwester.dict')  # store the dictionary, for future reference
print(dictionary)

In [ ]:
print(dictionary.token2id)

In [ ]:
new_doc = "Human computer interaction"
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec)  # the word "interaction" does not appear in the dictionary and is ignored

In [ ]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('../tmp/deerwester.mm', corpus)  # store to disk, for later use
pprint(corpus)

In [ ]:
class MyCorpus(object):
    def __iter__(self):
        for line in open('../data/mycorpus.txt'):
            # assume there's one document per line, tokens separated by whitespace
            yield dictionary.doc2bow(line.lower().split())

In [ ]:
corpus_memory_friendly = MyCorpus()  # doesn't load the corpus into memory!
print(corpus_memory_friendly)

In [ ]:
for vector in corpus_memory_friendly:  # load one vector into memory at a time
    print(vector)

In [ ]:
from six import iteritems

# collect statistics about all tokens
dictionary = corpora.Dictionary(line.lower().split() for line in open('../data/mycorpus.txt'))

# remove stop words and words that appear only once
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist
            if stopword in dictionary.token2id]
once_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq == 1]
dictionary.filter_tokens(stop_ids + once_ids)  # remove stop words and words that appear only once
dictionary.compactify()  # remove gaps in id sequence after words that were removed
print(dictionary)

In [ ]:
# create a toy corpus of 2 documents, as a plain Python list
corpus = [[(1, 0.5)], []]  # make one document empty, for the heck of it

corpora.MmCorpus.serialize('../tmp/corpus.mm', corpus)

In [ ]:
corpus = corpora.MmCorpus('../tmp/corpus.mm')

In [ ]:
print(corpus)

# one way of printing a corpus: load it entirely into memory
print(list(corpus)) 

# another way of doing it: print one document at a time, making use of the streaming interface
#for doc in corpus:
#    print(doc)

## Tutorial 2 : Topics and Transformations

In [ ]:
import os
from gensim import corpora, models, similarities

if (os.path.exists("../tmp/deerwester.dict")):
    dictionary = corpora.Dictionary.load('../tmp/deerwester.dict')
    corpus = corpora.MmCorpus('../tmp/deerwester.mm')
    print("Used files generated from first tutorial")
else:
    print("Please run first tutorial to generate data set")

In [ ]:
tfidf = models.TfidfModel(corpus) # step 1 -- initialize a model

In [ ]:
doc_bow = [(0, 1), (1, 1)]
print(tfidf[doc_bow]) # step 2 -- use the model to transform vectors

In [ ]:
corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf:
    print(doc)

In [ ]:
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2) # initialize an LSI transformation
corpus_lsi = lsi[corpus_tfidf] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

In [ ]:
for doc in corpus_lsi: # both bow->tfidf and tfidf->lsi transformations are actually executed here, on the fly
    print(doc)

In [ ]:
lsi.save('../tmp/model.lsi') # same for tfidf, lda, ...
lsi = models.LsiModel.load('../tmp/model.lsi')

## Tutorial 3 : Similarity Queries

In [ ]:
from gensim import corpora, models, similarities
dictionary = corpora.Dictionary.load('../tmp/deerwester.dict')
corpus = corpora.MmCorpus('../tmp/deerwester.mm') # comes from the first tutorial, "From strings to vectors"
print(corpus)

In [ ]:
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)

In [ ]:
doc = "Human computer interaction"
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow] # convert the query to LSI space
print(vec_lsi)

In [ ]:
index = similarities.MatrixSimilarity(lsi[corpus]) # transform corpus to LSI space and index it

In [ ]:
index.save('../tmp/deerwester.index')
index = similarities.MatrixSimilarity.load('../tmp/deerwester.index')

In [ ]:
sims = index[vec_lsi] # perform a similarity query against the corpus
print(list(enumerate(sims))) # print (document_number, document_similarity) 2-tuples

In [ ]:
sims = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims) # print sorted (document number, similarity score) 2-tuples